## Thank you for visit


#### In this notebook, I want to show Example how to calculate MAP@12 

#### If you like this notebook, upvote please 😉

### Version update (Feb 15, 2022)

- I receive message from @hervind and @t88take that tell me my mistake about competition metrics.

- So, I fixed this notebook.

- Thank you.

### Version update (Feb 16, 2022)

- I fixed this notebook again. 😂

- I saw some discussions and learned from previous released notebooks.

- If this will be useful for you, I'm happy too.

### Version update (Feb 26, 2022)

- I fixed this notebook again. 😂

- I dropped no purchase customer in valid term.

In [ ]:
import numpy as np
import pandas as pd
import gc
import os
import time
import random
from tqdm.auto import tqdm

In [ ]:
def visualize_df(df):
    print(df.shape)
    display(df.head())

In [ ]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
visualize_df(transactions_train)

In [ ]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
del sub['prediction']; gc.collect()
visualize_df(sub)

In [ ]:
# transactions_train['t_dat'].unique()[-7:]

# array(['2020-09-16', '2020-09-17', '2020-09-18', '2020-09-19',
#       '2020-09-20', '2020-09-21', '2020-09-22'], dtype=object)

In [ ]:
val_start_date = '2020-09-16'

train_data = transactions_train.query(f"t_dat < '{val_start_date}'").reset_index(drop=True)
valid_data = transactions_train.query(f"t_dat >= '{val_start_date}'").reset_index(drop=True)

visualize_df(train_data)
visualize_df(valid_data)

In [ ]:
train_unq = train_data.groupby('customer_id')['article_id'].apply(list).reset_index()
train_unq['valid_pred'] = train_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
visualize_df(train_unq)

In [ ]:
valid_unq = valid_data.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_unq['valid_true'] = valid_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
visualize_df(valid_unq)

In [ ]:
merged = pd.merge(sub, train_unq, on='customer_id', how='left').fillna('')
merged = pd.merge(merged, valid_unq, on='customer_id', how='left').fillna('')

del merged['article_id_x'], merged['article_id_y']; gc.collect()
merged.head()

In [ ]:
# remove customers who made no purchases 

merged = merged[merged['valid_true']!=''].reset_index(drop=True)
print(merged.shape)
merged.head()

In [ ]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py


def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # remove this case in advance
    # if not actual:
    #     return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
tqdm.pandas()

mapk(
    merged['valid_true'].map(lambda x: x.split()), 
    merged['valid_pred'].map(lambda x: x.split()), 
    k=12
)

#### CV is 0.0029

#### Actual LB score is 0.001XX

In [ ]:
sub = merged[['customer_id', 'valid_pred']].copy()
sub.columns = ['customer_id', 'prediction']
print(sub.shape)

sub.to_csv('submission.csv', index=False)